In [16]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from simple_colors import *
import plotly.figure_factory as ff
import plotly as py
from itertools import cycle, islice
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import warnings
warnings.filterwarnings("ignore")
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import psycopg2 as pg
import anvil.server
anvil.server.connect("B7FGSYFNMM55I6K2BTYFDWHJ-3TSAKIF5IBLXJBVO")

In [18]:
@anvil.server.callable
def predict_state(state):
    while True:
        state = state.upper()
        try:
            df = 'senate_model'
            engine = pg.connect("dbname='postgres' user='postgres' host='elections.c8frdoyd7uok.us-east-2.rds.amazonaws.com' port='5432' password='Flowers89!'")
            df = pd.read_sql(f'select * from {df}', con=engine)
        except:
            df = pd.read_csv('https://hieppham.s3.us-east-2.amazonaws.com/Final_project/senate_dataset.csv')
        df = df.dropna()
        df= df[df['Results'] != 0]
        df.rename(columns = {'party_simplified' : 'winning_party'},inplace = True)
        try:
            #state = input("Please enter the STATE to predict: ").strip().upper()   
            df = df.loc[df.state_po == state]   
            #print(red("Typo! Please try again."))
            columns = ['state_po', '_year', 'candidatevotes', 'totalvotes','winning_party']
            df = df[columns]    
            #print(f"Here's {state} DataFrame {df}"'\n')
            print(df)

            #FIRST PREDICTION
            ##the last column is our label
            y_train = df.totalvotes.values
            #drop last column of data
            X_train = df.drop(['winning_party', 'state_po'], axis=1)
            #drop first colum of data
            X_test = df.drop(['winning_party', 'state_po', '_year'], axis=1)
            from sklearn.ensemble import RandomForestRegressor
            model = RandomForestRegressor(max_depth=5, random_state=1, n_estimators=1000).fit(X_train, y_train)
            y_pred = model.predict(X_train)
            new_pred = y_pred[-1]
            prediction = pd.DataFrame({'State':df.state_po,'Last_Election':df.totalvotes, 'Next_Election_Pred': y_pred.astype(int)}, index=None)
            prediction = prediction.iloc[[-1]]
            print (prediction.to_string(index=False))

            y = df.totalvotes.values
            #drop last column of data
            X = df.drop(['winning_party', 'state_po'], axis=1)
            X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, test_size=0.01) #0.2 means only 20% sample
            X_train.shape, X_test.shape, y_train.shape, y_test.shape
            model = RandomForestRegressor(max_depth=5, random_state=1, n_estimators=1000).fit(X_train, y_train)
            y_pred = model.predict(X_train).astype(int)
            #print(blue(f"Predict_Score of {state}: {model.score(X_train, y_train)}"))
            total_votes_pred = y_pred[-1]
            total_votes_pred

                #SECOND PREDICTION
            df.winning_party = df. winning_party.replace({'DEMOCRAT': 1, 'REPUBLICAN': 2, 'LIBERTARIAN': 3, 'OTHER': 4})
            # the last column is our label
            y = df.winning_party.values
            #drop last column of data
            X = df.drop(['winning_party', 'state_po'], axis=1)
            X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, test_size=0.01) #0.2 means only 20% sample
            #print(X_train.shape)
            model2 = RandomForestRegressor(max_depth=5, random_state=1, n_estimators=1000).fit(X_train, y_train)
            y_pred = model2.predict(X_train)
            #print(len(y_pred))
            print(blue(f"Predict_Score of {state}: {model2.score(X_train, y_train)}"))
            #print(y_pred)
        except ValueError:
            print(red("Typo! Please try again."'\n'))
            break

        lst1 = [x for x in y_train[-20:] if x == 1] 
        dem = len(lst1)
        lst2 = [x for x in y_train[-20:] if x == 2] 
        rep = len(lst2)
        lst3 = [x for x in y_train[-20:] if (x ==3)] 
        lib = len(lst3)
        lst4 = [x for x in y_train[-20:] if (x != 1 and x !=2 and x !=3)] 
        other = len(lst4)
        if other > dem and other > rep:
            print_pred = f"Predicts OTHER will win in {state}"
            print(green(f"Predicts OTHER will win in {state} next Election."))
        elif dem > rep and dem >= other:      
            print_pred = f"Predicts SENATE DEMOCRAT will win in {state}"
            print(blue(f"Predicts SENATE DEMOCRAT will win in {state} next Election."))
        elif rep >= dem and rep >= other:
            print_pred = f"Predicts SENATE REPUBLICAN will win in {state}"
            print(red(f"Predicts SENATE REPUBLICAN will win in {state} next Election."))
        return print_pred
    